# 강아지(바론) LoRA 학습 데이터 생성기

## 목적
어둡고 불안정한 동화 세계관 속 **인간의 영혼을 알아보는 강아지 바론**의 행동 묘사 데이터를 생성한다.  
LoRA 미세조정 학습용 `{"input": ..., "output": ...}` JSONL 형식.

## 핵심 원칙
- **출력 형식**: 3인칭 행동 묘사 (`바론이 꼬리를 흔들며 달려옵니다.`)
- **바론은 말하지 않음**: 직접 대사 금지, 행동과 신체 언어로만 표현
- **인간의 영혼 탐지**: 플레이어의 인간성 수준에 민감하게 반응
- **rule-base와 역할 분리**: LoRA는 행동 묘사 어휘와 상황 반응을 학습, 호감도 모디파이어는 후처리에서 처리

## 의미 축 (6개)
| 축 | input 방향 | output 귀결 |
|---|---|---|
| 친근한 접촉 | 머리 쓰다듬기, 손 내밀기 | 꼬리 흔들기, 핥기, 낑낑 |
| 먹이 주기 | 간식 건네기 | 흥분하여 달려옴, 받아먹음 |
| 가족 사진 | 가족 사진 보여주기 | 특별한 반응, 정원 방향 유인 |
| 무시/방치 | 외면, 지나침 | 슬픈 낑낑, 졸졸 따라옴 |
| 위협/큰소리 | 소리 지름, 위협 | 짖기, 물러남, 경계 |
| 탐색 안내 | (높은 호감도) 탐색 행동 | 아이템 있는 방향으로 유인 |

## 생성 수량
- 세트당 400개 × 2세트 = 총 800개
- input 길이 분포: 짧은(35%), 중간(50%), 긴(15%)

In [ ]:
import json
import random
from pathlib import Path
from typing import List, Dict, Tuple
from collections import Counter

## 1. 의미 축별 input/output 풀 정의

In [ ]:
# ============================================================
# 의미 축별 input/output 풀
#
# input: 플레이어의 행동 설명 (게임 내 서술)
# output: 바론의 행동 묘사 (3인칭 서술체, ~합니다/~입니다)
# ============================================================

SEMANTIC_AXES: Dict[str, dict] = {

    # ── 축 1: 친근한 접촉 ──────────────────────────────────────
    # output: 우호적 신체 반응 — 꼬리 흔들기, 핥기, 달려옴
    "friendly_touch": {
        "short": [
            "바론의 머리를 쓰다듬었다.",
            "손을 내밀었다.",
            "바론에게 다가갔다.",
            "바론을 안아줬다.",
            "등을 쓰다듬었다.",
            "바론의 이름을 불렀다.",
            "옆에 앉았다.",
        ],
        "medium": [
            "조심스럽게 바론의 귀 뒤를 긁어줬다.",
            "바닥에 앉아 바론과 눈을 맞췄다.",
            "천천히 손을 내밀어 바론이 냄새를 맡게 했다.",
            "바론의 등에 손을 올렸다.",
            "낮은 목소리로 바론을 불렀다.",
            "바론 앞에 쪼그리고 앉아 바라봤다.",
            "바론의 발을 가볍게 잡아봤다.",
            "바론이 있는 쪽으로 천천히 걸어갔다.",
        ],
        "long": [
            "바론 옆에 무릎을 꿇고 앉아 천천히 머리를 쓰다듬으며 이름을 불렀다.",
            "바닥에 앉아 바론이 다가올 때까지 기다렸다가 조심스럽게 귀를 긁어줬다.",
            "바론과 눈높이를 맞추고 낮은 목소리로 말을 걸었다.",
        ],
        "outputs": [
            "바론이 꼬리를 세차게 흔들며 달려옵니다.",
            "바론이 당신의 손을 핥습니다.",
            "바론이 기쁜 듯 낑낑거리며 몸을 비빕니다.",
            "바론이 앞발을 살짝 들어 반깁니다.",
            "바론이 꼬리를 흔들며 코를 비볐습니다.",
            "바론이 당신의 발 옆에 엎드립니다.",
            "바론이 낑낑거리며 뒷발로 일어섭니다.",
            "바론이 혀를 내밀며 당신의 볼을 핥습니다.",
            "바론이 꼬리를 흔들며 제자리를 빙글빙글 돕니다.",
            "바론이 뒹굴며 배를 보여줍니다.",
            "바론이 당신 품에 머리를 묻습니다.",
            "바론이 눈을 가늘게 뜨고 가르랑거리듯 낑낑댑니다.",
            "바론이 꼬리를 흔들며 옆에 바짝 붙어 앉습니다.",
            "바론이 당신의 손을 핥으며 눈을 감습니다.",
            "바론이 기쁘게 짧은 소리를 내며 맴돕니다.",
        ],
    },

    # ── 축 2: 먹이 주기 ──────────────────────────────────────────
    # output: 흥분 + 먹이 수령 행동
    "feeding": {
        "short": [
            "간식을 꺼냈다.",
            "음식을 내밀었다.",
            "먹을 걸 줬다.",
            "고기를 내밀었다.",
            "비스킷을 건넸다.",
        ],
        "medium": [
            "주머니에서 간식을 꺼내 바론 앞에 내밀었다.",
            "손바닥 위에 음식 조각을 올려 바론에게 보여줬다.",
            "바론의 앞에 먹이 그릇을 놓았다.",
            "작은 고기 조각을 바닥에 던져줬다.",
            "간식을 손 위에 올려 바론이 먹도록 했다.",
            "부엌에서 가져온 음식을 바론에게 줬다.",
        ],
        "long": [
            "주머니에 넣어뒀던 비스킷을 꺼내 바론 앞에서 보여주며 이름을 불렀다.",
            "바론이 배가 고파 보여 손에 음식을 올려 천천히 내밀었다.",
        ],
        "outputs": [
            "바론이 흥분하며 앞발로 땅을 탁탁 칩니다.",
            "바론이 달려와 간식을 받아먹습니다.",
            "바론이 꼬리를 흔들며 냄새를 킁킁 맡습니다.",
            "바론이 혀로 손바닥을 핥으며 간식을 먹습니다.",
            "바론이 두 눈을 반짝이며 달려옵니다.",
            "바론이 기쁘게 맴돌다가 받아먹습니다.",
            "바론이 앞발을 세우며 간식을 향해 달려듭니다.",
            "바론이 조심스럽게 손에서 받아먹습니다.",
            "바론이 꼬리를 격렬히 흔들며 먹이를 기다립니다.",
            "바론이 빠르게 핥아 삼킵니다.",
            "바론이 맛있는 듯 낑낑거리며 더 달라는 시늉을 합니다.",
            "바론이 그릇 주변을 빙빙 돌다가 먹기 시작합니다.",
            "바론이 받아먹고는 꼬리를 흔들며 감사를 표합니다.",
            "바론이 냄새를 먼저 맡아보고 게걸스럽게 먹습니다.",
            "바론이 고개를 들어 당신을 바라본 뒤 받아먹습니다.",
        ],
    },

    # ── 축 3: 가족 사진 반응 ───────────────────────────────────
    # output: 특별하고 강렬한 반응 — 진짜 기억에 반응
    "family_photo": {
        "short": [
            "가족 사진을 꺼냈다.",
            "오래된 사진을 보여줬다.",
            "사진을 펼쳤다.",
            "가족 사진을 들었다.",
        ],
        "medium": [
            "주머니에서 가족 사진을 꺼내 바론 앞에 내밀었다.",
            "발견한 가족 사진을 바론에게 보여줬다.",
            "오래된 가족 사진을 펼쳐 바론이 냄새를 맡게 했다.",
            "바론 앞에 사진을 내려놓았다.",
            "예전 가족 사진을 바론의 코 가까이 가져갔다.",
        ],
        "long": [
            "숨겨뒀던 가족 사진을 꺼내 바론 앞에서 펼쳐 보이며 이름들을 불렀다.",
            "다락에서 발견한 낡은 가족 사진을 조심스럽게 바론 앞에 내밀었다.",
        ],
        "outputs": [
            "바론이 코를 킁킁거리며 사진에 가까이 다가옵니다.",
            "바론이 낑낑거리며 정원 쪽으로 향하기 시작합니다.",
            "바론이 사진 냄새를 맡고는 앞발로 땅을 긁습니다.",
            "바론이 갑자기 정원 방향으로 달려가 멈춥니다.",
            "바론이 깊이 냄새를 맡고는 당신을 뚫어지게 바라봅니다.",
            "바론이 짧게 짖고 정원 방향으로 고개를 돌립니다.",
            "바론이 흥분하며 정원 쪽으로 앞발로 땅을 파는 시늉을 합니다.",
            "바론이 코를 땅에 대고 냄새를 추적합니다.",
            "바론이 눈을 크게 뜨고 당신과 사진을 번갈아 봅니다.",
            "바론이 특별한 반응을 보이며 특정 방향으로 걷기 시작합니다.",
            "바론이 낮게 낑낑거리며 꼬리를 세우고 앞서 걷습니다.",
            "바론이 사진을 핥으려 하다가 정원 쪽을 바라봅니다.",
            "바론이 흥분된 기색으로 당신 주위를 빙빙 돌다가 정원으로 향합니다.",
            "바론이 잠시 멈추더니 조심스럽게 꽃밭 쪽으로 걷기 시작합니다.",
            "바론이 사진 냄새를 오래 맡다가 뭔가를 기억한 듯 앞을 향해 달립니다.",
        ],
    },

    # ── 축 4: 무시/방치 ─────────────────────────────────────────
    # output: 슬픈 반응 — 낑낑거림, 졸졸 따라옴
    "indifference": {
        "short": [
            "바론을 무시했다.",
            "그냥 지나쳤다.",
            "바론을 외면했다.",
            "눈을 마주치지 않았다.",
            "바론에게 등을 돌렸다.",
        ],
        "medium": [
            "바론이 다가왔지만 그냥 지나쳤다.",
            "바론의 낑낑 소리를 무시하고 계속 걸었다.",
            "바론을 보지 않고 다른 곳으로 시선을 돌렸다.",
            "바론이 발에 비볐지만 신경 쓰지 않았다.",
            "바론에게 아무런 반응도 보이지 않았다.",
            "바론이 다가와도 말을 걸지 않았다.",
            "바론에게서 멀리 떨어진 곳에 앉았다.",
        ],
        "long": [
            "바론이 꼬리를 흔들며 다가왔지만 다른 생각에 빠져 그냥 지나쳤다.",
            "바론이 옆에 앉아 기다렸지만 계속 다른 데 신경을 쓰며 무시했다.",
        ],
        "outputs": [
            "바론이 낮게 낑낑거리며 당신 뒤를 졸졸 따라옵니다.",
            "바론이 슬픈 눈으로 당신을 바라봅니다.",
            "바론이 꼬리를 축 늘어뜨리고 제자리에 앉습니다.",
            "바론이 조용히 당신 발 옆에 엎드립니다.",
            "바론이 짧게 낑낑거리다가 바닥에 머리를 내려놓습니다.",
            "바론이 거리를 유지하며 조용히 당신을 지켜봅니다.",
            "바론이 한숨 쉬듯 깊은 숨을 내쉬고 엎드립니다.",
            "바론이 꼬리를 느릿느릿 흔들며 당신 시야 안에 머뭅니다.",
            "바론이 발걸음을 멈추고 당신이 돌아보길 기다립니다.",
            "바론이 낑낑거리다 포기하고 제자리로 돌아갑니다.",
            "바론이 쓸쓸히 혼자 구석으로 걸어갑니다.",
            "바론이 고개를 숙이고 엎드린 채 당신을 바라봅니다.",
            "바론이 슬픈 눈빛으로 꼬리를 약하게 흔들다 멈춥니다.",
            "바론이 발로 당신 발을 살짝 건드리고는 물러납니다.",
            "바론이 작게 낑낑거리며 털을 핥기 시작합니다.",
        ],
    },

    # ── 축 5: 위협/큰소리 ────────────────────────────────────────
    # output: 짖기, 으르렁, 물러남
    "threat_response": {
        "short": [
            "큰 소리를 질렀다.",
            "바론에게 소리쳤다.",
            "발을 굴렀다.",
            "뭔가를 집어 들었다.",
            "위협적으로 다가갔다.",
            "바론에게 발길질했다.",
        ],
        "medium": [
            "바론에게 화를 내며 소리쳤다.",
            "날카롭게 바론을 꾸짖었다.",
            "빗자루를 들어 바론을 쫓았다.",
            "갑자기 큰 소리를 냈다.",
            "손을 빠르게 움직여 놀라게 했다.",
            "위협적으로 몸을 바론 쪽으로 기울였다.",
            "바론에게 물건을 던졌다.",
        ],
        "long": [
            "바론이 짖는 것에 화가 나서 큰 소리로 꾸짖으며 손을 휘둘렀다.",
            "바론이 다가오는 것이 싫어서 위협적인 몸짓과 함께 소리를 질렀다.",
        ],
        "outputs": [
            "바론이 날카롭게 짖으며 뒷걸음칩니다.",
            "바론이 낮게 으르렁거리며 이빨을 드러냅니다.",
            "바론이 등털을 세우고 당신을 경계합니다.",
            "바론이 빠르게 물러나 구석에 웅크립니다.",
            "바론이 격렬하게 짖다가 방 구석으로 물러납니다.",
            "바론이 낮은 자세로 으르렁거리며 당신을 주시합니다.",
            "바론이 짖으며 문 쪽으로 도망칩니다.",
            "바론이 겁먹은 듯 짖다가 등을 돌립니다.",
            "바론이 꼬리를 내리고 몸을 낮추며 뒷걸음칩니다.",
            "바론이 날카롭게 짖으며 당신과 거리를 유지합니다.",
            "바론이 이빨을 보이며 낮은 소리를 냅니다.",
            "바론이 잔뜩 긴장하여 온몸을 떱니다.",
            "바론이 거리를 두고 계속해서 짖습니다.",
            "바론이 두려움에 낑낑대며 구석으로 물러납니다.",
            "바론이 경계하며 눈을 떼지 않고 뒷걸음칩니다.",
        ],
    },

    # ── 축 6: 탐색 안내 ─────────────────────────────────────────
    # output: 아이템/위치 방향으로 유인하는 행동
    "item_guide": {
        "short": [
            "바론이 어디론가 달려갔다.",
            "바론이 멈춰 섰다.",
            "바론이 바닥을 파기 시작했다.",
            "바론이 한 방향을 계속 바라봤다.",
            "바론이 특이한 행동을 했다.",
        ],
        "medium": [
            "바론이 뭔가를 발견한 듯 특정 방향으로 달려갔다.",
            "바론이 꽃밭 근처에서 바닥을 긁기 시작했다.",
            "바론이 오래된 나무 아래에서 땅을 파기 시작했다.",
            "바론이 멈추더니 당신에게 따라오라는 듯 짖었다.",
            "바론이 특정 구석 방향으로 계속 코를 향했다.",
            "바론이 창고 문 앞에서 앞발로 문을 긁었다.",
            "바론이 뭔가 냄새를 맡으며 앞서 걸었다.",
        ],
        "long": [
            "높은 호감도 상태에서 바론이 갑자기 정원 쪽으로 달려가더니 특정 지점에서 땅을 파기 시작했다.",
            "바론이 당신을 바라보고 짖은 뒤 창고 방향으로 달려가다 멈추며 당신이 따라오는지 확인했다.",
            "바론이 꽃밭 특정 지점에서 앞발로 땅을 계속 파더니 뭔가를 물어 올렸다.",
        ],
        "outputs": [
            "바론이 꽃밭 한 지점을 앞발로 긁습니다.",
            "바론이 정원 구석으로 달려가 당신을 기다립니다.",
            "바론이 특정 방향으로 코를 집중합니다.",
            "바론이 짖으며 창고 방향으로 달려갑니다.",
            "바론이 땅을 파다가 낡은 천 조각을 물어옵니다.",
            "바론이 당신을 보며 짖은 뒤 정원 쪽으로 달립니다.",
            "바론이 나무 아래에서 앞발로 흙을 긁습니다.",
            "바론이 특정 장소 앞에서 멈추고 냄새를 맡습니다.",
            "바론이 낮게 낑낑거리며 창고 문 앞에서 기다립니다.",
            "바론이 바닥을 파다가 당신을 돌아봅니다.",
            "바론이 오래된 나무 뿌리 근처에서 코를 킁킁댑니다.",
            "바론이 뭔가를 발견한 듯 흥분하며 짖습니다.",
            "바론이 꽃밭 깊숙이 파고들며 흙을 뒤집습니다.",
            "바론이 무언가를 물어올 것처럼 땅을 격렬히 팝니다.",
            "바론이 당신과 땅을 번갈아 보며 낑낑거립니다.",
        ],
    },
}

print(f"정의된 의미 축: {list(SEMANTIC_AXES.keys())}")
for axis, data in SEMANTIC_AXES.items():
    total_inputs = len(data['short']) + len(data['medium']) + len(data['long'])
    print(f"  {axis}: input {total_inputs}개 / output {len(data['outputs'])}개")

## 2. 의미 대비 쌍 정의

In [ ]:
CONTRAST_PAIRS: List[Dict] = [
    # 친근함 ↔ 위협
    {
        "pair": [
            {"axis": "friendly_touch",  "input": "바론의 머리를 쓰다듬었다."},
            {"axis": "threat_response", "input": "바론에게 소리쳤다."},
        ]
    },
    {
        "pair": [
            {"axis": "friendly_touch",  "input": "바닥에 앉아 바론과 눈을 맞췄다."},
            {"axis": "threat_response", "input": "위협적으로 몸을 바론 쪽으로 기울였다."},
        ]
    },
    # 먹이 ↔ 무시
    {
        "pair": [
            {"axis": "feeding",      "input": "간식을 꺼냈다."},
            {"axis": "indifference", "input": "바론을 무시했다."},
        ]
    },
    {
        "pair": [
            {"axis": "feeding",      "input": "손바닥 위에 음식 조각을 올려 바론에게 보여줬다."},
            {"axis": "indifference", "input": "바론이 다가왔지만 그냥 지나쳤다."},
        ]
    },
    # 가족 사진 ↔ 위협
    {
        "pair": [
            {"axis": "family_photo",  "input": "가족 사진을 꺼냈다."},
            {"axis": "threat_response", "input": "큰 소리를 질렀다."},
        ]
    },
    # 탐색 안내 ↔ 무시
    {
        "pair": [
            {"axis": "item_guide",   "input": "바론이 멈추더니 당신에게 따라오라는 듯 짖었다."},
            {"axis": "indifference", "input": "바론의 낑낑 소리를 무시하고 계속 걸었다."},
        ]
    },
    # 친근함 ↔ 무시
    {
        "pair": [
            {"axis": "friendly_touch", "input": "낮은 목소리로 바론을 불렀다."},
            {"axis": "indifference",   "input": "바론에게서 멀리 떨어진 곳에 앉았다."},
        ]
    },
]

print(f"정의된 의미 대비 쌍: {len(CONTRAST_PAIRS)}개")

## 3. 변형 함수 (다양성 확보)

In [ ]:
def vary_output(text: str) -> str:
    """output 행동 묘사에 자연스러운 변형을 추가한다.
    3인칭 묘사 형식과 의미 방향은 보존.
    """
    result = text

    # 시간 부사 추가 (15%)
    if random.random() < 0.15:
        time_adverbs = ["잠시 후 ", "곧바로 ", "천천히 ", "갑자기 "]
        # "바론이" 앞에 삽입
        if result.startswith("바론이"):
            result = random.choice(time_adverbs) + result

    # 추가 묘사 (15%)
    if random.random() < 0.15:
        additions = [
            " 그리고 당신을 바라봅니다.",
            " 눈을 당신에게서 떼지 않습니다.",
            " 잠시 멈추고 주위를 살핍니다.",
        ]
        if result.endswith(".") and not any(result.endswith(a.strip()) for a in additions):
            result = result[:-1] + random.choice(additions)

    return result


def vary_input(text: str) -> str:
    """input(플레이어 행동 서술)에 자연스러운 변형을 추가한다."""
    result = text

    # 과거형/현재형 변형 (20%)
    if random.random() < 0.2:
        variants = [
            ("했다.", ["했다.", "했습니다.", "했어요."]),
            ("갔다.", ["갔다.", "갔습니다.", "향했다."]),
            ("줬다.", ["줬다.", "건넸다.", "내밀었다."]),
        ]
        for original, alts in variants:
            if result.endswith(original):
                result = result[:-len(original)] + random.choice(alts)
                break

    return result


print("=== output 변형 테스트 ===")
test_output = "바론이 꼬리를 세차게 흔들며 달려옵니다."
for i in range(5):
    print(f"  [{i+1}] {vary_output(test_output)}")

print("\n=== input 변형 테스트 ===")
test_input = "바론의 머리를 쓰다듬었다."
for i in range(5):
    print(f"  [{i+1}] {vary_input(test_input)}")

## 4. 데이터 생성 엔진

In [ ]:
def select_input_by_length(axis_data: dict) -> Tuple[str, str]:
    roll = random.random()
    if roll < 0.35:
        category = "short"
    elif roll < 0.85:
        category = "medium"
    else:
        category = "long"
    text = random.choice(axis_data[category])
    return text, category


def generate_pair(axis_name: str) -> dict:
    axis_data = SEMANTIC_AXES[axis_name]
    input_text, length_cat = select_input_by_length(axis_data)
    input_text = vary_input(input_text)
    output_text = random.choice(axis_data["outputs"])
    output_text = vary_output(output_text)
    return {
        "input": input_text,
        "output": output_text,
        "_axis": axis_name,
        "_length": length_cat,
    }


def generate_contrast_pair(pair_def: dict) -> List[dict]:
    results = []
    for item in pair_def["pair"]:
        axis_name = item["axis"]
        axis_data = SEMANTIC_AXES[axis_name]
        input_text = vary_input(item["input"])
        output_text = random.choice(axis_data["outputs"])
        output_text = vary_output(output_text)
        results.append({
            "input": input_text,
            "output": output_text,
            "_axis": axis_name,
            "_length": "contrast",
        })
    return results


def generate_dataset(num_samples: int = 400, seed: int = 42) -> List[dict]:
    random.seed(seed)
    data = []
    axes = list(SEMANTIC_AXES.keys())

    for _ in range(2):
        for pair_def in CONTRAST_PAIRS:
            pairs = generate_contrast_pair(pair_def)
            data.extend(pairs)

    remaining = num_samples - len(data)
    per_axis = remaining // len(axes)
    leftover = remaining % len(axes)

    for i, axis_name in enumerate(axes):
        count = per_axis + (1 if i < leftover else 0)
        for _ in range(count):
            pair = generate_pair(axis_name)
            data.append(pair)

    random.shuffle(data)
    return data


print("데이터 생성 함수 정의 완료.")

## 5. 데이터 생성 및 저장

In [ ]:
def validate_dataset(data: List[dict]) -> dict:
    stats = {
        "total": len(data),
        "axis_dist": Counter(),
        "length_dist": Counter(),
        "input_lengths": [],
        "output_lengths": [],
        "duplicates": 0,
    }
    seen_inputs = set()
    for item in data:
        stats["axis_dist"][item["_axis"]] += 1
        stats["length_dist"][item["_length"]] += 1
        stats["input_lengths"].append(len(item["input"]))
        stats["output_lengths"].append(len(item["output"]))
        if item["input"] in seen_inputs:
            stats["duplicates"] += 1
        seen_inputs.add(item["input"])
    return stats


set1 = generate_dataset(num_samples=400, seed=42)
set2 = generate_dataset(num_samples=400, seed=1337)

stats1 = validate_dataset(set1)
print(f"세트 1: 총 {stats1['total']}개, 중복 {stats1['duplicates']}개")
for axis, count in sorted(stats1['axis_dist'].items()):
    print(f"  {axis}: {count}개")

print()
stats2 = validate_dataset(set2)
print(f"세트 2: 총 {stats2['total']}개, 중복 {stats2['duplicates']}개")

In [ ]:
def save_jsonl(data: List[dict], output_path: str) -> None:
    output_file = Path(output_path)
    output_file.parent.mkdir(parents=True, exist_ok=True)
    with open(output_file, "w", encoding="utf-8") as f:
        for item in data:
            clean = {"input": item["input"], "output": item["output"]}
            f.write(json.dumps(clean, ensure_ascii=False) + "\n")
    print(f"저장 완료: {output_path} ({len(data)}개)")


OUTPUT_DIR = Path("../data/dog_baron")

save_jsonl(set1, str(OUTPUT_DIR / "dog_baron_dialogue_00.jsonl"))
save_jsonl(set2, str(OUTPUT_DIR / "dog_baron_dialogue_01.jsonl"))

combined = set1 + set2
random.shuffle(combined)
save_jsonl(combined, str(OUTPUT_DIR / "dog_baron_dialogue_combined.jsonl"))
print(f"\n합본 저장 완료: {len(combined)}개")

print("\n=== 의미 축별 샘플 ===")
axis_names_kr = {
    "friendly_touch":  "친근한 접촉",
    "feeding":         "먹이 주기",
    "family_photo":    "가족 사진 반응",
    "indifference":    "무시/방치",
    "threat_response": "위협/큰소리",
    "item_guide":      "탐색 안내",
}
for axis in SEMANTIC_AXES.keys():
    axis_items = [item for item in combined if item["_axis"] == axis]
    samples = random.sample(axis_items, min(2, len(axis_items)))
    print(f"\n--- {axis_names_kr.get(axis, axis)} ---")
    for s in samples:
        print(f"  플레이어: {s['input']}")
        print(f"  바론:     {s['output']}")
        print()